In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
HOPSWORKS_PROJECT_NAME = 'bike_sharing_demand'

In [5]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [6]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

File 2022 was already in local storage
descargado año 2023


In [7]:
print(f'{len(rides)=}')

len(rides)=4612559


In [15]:
rides

,pickup_datetime,pickup_location_id,pickup_hour
0,2022-01-16 14:58:42,5,2022-01-16 14:00:00
1,2022-01-16 17:26:27,51,2022-01-16 17:00:00
2,2022-01-16 16:51:12,161,2022-01-16 16:00:00
3,2022-01-16 15:58:01,210,2022-01-16 15:00:00
4,2022-01-16 23:40:09,215,2022-01-16 23:00:00
...,...,...,...
1689749,2023-01-10 08:11:05,116,2023-01-10 08:00:00
1689750,2023-05-31 17:38:25,14,2023-05-31 17:00:00
1689751,2023-04-17 20:56:45,186,2023-04-17 20:00:00
1689752,2023-01-31 16:23:13,75,2023-01-31 16:00:00


In [8]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 350/350 [00:10<00:00, 34.39it/s]


In [16]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2022-01-01 00:00:00,1,9
1,2022-01-01 01:00:00,1,9
2,2022-01-01 02:00:00,1,9
3,2022-01-01 03:00:00,0,9
4,2022-01-01 04:00:00,1,9
...,...,...,...
5107195,2023-08-31 19:00:00,0,507
5107196,2023-08-31 20:00:00,0,507
5107197,2023-08-31 21:00:00,0,507
5107198,2023-08-31 22:00:00,0,507


In [9]:
import hopsworks

print(hopsworks.__version__)

3.2.0


In [10]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/100501


In [11]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [12]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [13]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [14]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/5107200 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/100501/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x28103e713d0>, None)